In [1]:
import geopandas as gpd
import os
import zipfile as zf
import geopandas
from shapely.geometry import shape
import gzip 
import urllib.request
import xarray as xr
import io
import shapefile
import pandas as pd
import json

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [138]:
df = pd.read_pickle("GDP_temperature_anomalies_1947_USA.pkl")

In [91]:
df

,time,timeMin,timeMax,gdp
0,1947-01-15,-6.66,4.33,2033.061000
1,1947-02-15,-3.48,3.51,2031.253667
2,1947-03-15,-3.64,4.59,2029.446333
3,1947-04-15,-1.90,1.61,2027.639000
4,1947-05-15,-2.55,2.46,2026.243333
...,...,...,...,...
887,2020-12-15,-0.27,5.58,18989.854000
888,2021-01-15,0.48,7.10,19087.568000
889,2021-02-15,-4.14,1.27,NaN
890,2021-03-15,-1.28,5.49,NaN


In [139]:
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df = df.set_index(df['time'])
df = df.sort_index()
df =  df.drop(["time","index","gdp"], axis =1)

In [93]:
df

,timeMin,timeMax
time,,
1947-01-15,-6.66,4.33
1947-02-15,-3.48,3.51
1947-03-15,-3.64,4.59
1947-04-15,-1.90,1.61
1947-05-15,-2.55,2.46
...,...,...
2020-12-15,-0.27,5.58
2021-01-15,0.48,7.10
2021-02-15,-4.14,1.27


In [97]:
892*0.75

669.0

In [73]:
df

,timeMin,timeMax
time,,
1980-01-15,0.662583,0.124825
1980-02-15,0.568332,0.467742
1980-03-15,0.680490,0.229313
1980-04-15,0.681433,0.214586
1980-05-15,0.672008,0.168303
...,...,...
2020-12-15,0.786993,0.392006
2021-01-15,0.857681,0.498597
2021-02-15,0.422243,0.089762


In [140]:
train,test = df[1:669], df[669:] 

In [99]:
train

,timeMin,timeMax
time,,
1947-02-15,-3.48,3.51
1947-03-15,-3.64,4.59
1947-04-15,-1.90,1.61
1947-05-15,-2.55,2.46
1947-06-15,-1.95,1.63
...,...,...
2002-05-15,-2.23,2.51
2002-06-15,-0.40,2.27
2002-07-15,-0.47,3.18


Scaling dataset

In [141]:
pd.options.mode.chained_assignment = None  # default='warn'
min_max_scaler = MinMaxScaler()
train[["timeMin", "timeMax"]] = min_max_scaler.fit_transform(train[["timeMin", "timeMax"]])

,timeMin,timeMax
time,,
1947-02-15,0.551935,0.294813
1947-03-15,0.535642,0.365725
1947-04-15,0.712831,0.170059
1947-05-15,0.646639,0.225870
1947-06-15,0.707739,0.171372
...,...,...
2002-05-15,0.679226,0.229153
2002-06-15,0.865580,0.213395
2002-07-15,0.858452,0.273145


In [142]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [143]:
n_past = 50
n_future = 1
n_features = 2

In [144]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [145]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#padded_shapes = ([618], ())#None.
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "functional_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_36 (LSTM)                  [(None, 100), (None, 41200       input_13[0][0]                   
__________________________________________________________________________________________________
repeat_vector_12 (RepeatVector) (None, 1, 100)       0           lstm_36[0][0]                    
__________________________________________________________________________________________________
lstm_37 (LSTM)                  (None, 1, 100)       80400       repeat_vector_12[0][0]           
                                                                 lstm_36[0][1]        

In [146]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

618
173
618
173


In [147]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#padded_shapes = ([618], ()) #THIS IS TO GET RID OF :WARNING:tensorflow:10 out of the last 11 calls to <function train_step at 0x7f4825f6d400> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different ecc
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_38 (LSTM)                  [(None, 50, 100), (N 41200       input_14[0][0]                   
__________________________________________________________________________________________________
lstm_39 (LSTM)                  [(None, 100), (None, 80400       lstm_38[0][0]                    
__________________________________________________________________________________________________
repeat_vector_13 (RepeatVector) (None, 1, 100)       0           lstm_39[0][0]                    
______________________________________________________________________________________

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])

Epoch 1/25


In [135]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

In [136]:
pred_e1d1

array([[[0.69402474, 0.55427516]],

       [[0.90368176, 0.722606  ]],

       [[0.98743165, 0.74354273]],

       [[0.88307995, 0.6059508 ]],

       [[0.61248887, 0.61322665]],

       [[0.6922094 , 0.6303461 ]],

       [[0.8573218 , 0.4716885 ]],

       [[0.96816164, 0.31405348]],

       [[0.963007  , 0.331959  ]],

       [[1.1178888 , 0.32110396]],

       [[1.1498663 , 0.3217109 ]],

       [[1.1470342 , 0.40153098]],

       [[1.1902058 , 0.64543086]],

       [[1.2134261 , 0.7428694 ]],

       [[1.2217888 , 0.6396523 ]],

       [[1.2018126 , 0.5108057 ]],

       [[1.1783117 , 0.47837204]],

       [[1.1131291 , 0.38792443]],

       [[1.0276953 , 0.26319128]],

       [[1.0121039 , 0.18888946]],

       [[0.93679655, 0.08318805]],

       [[0.8119676 , 0.04799985]],

       [[0.7477959 , 0.052125  ]],

       [[0.40768644, 0.02164986]],

       [[0.4191468 , 0.12364566]],

       [[0.29526246, 0.24879251]],

       [[0.22502398, 0.21044147]],

       [[0.36902863, 0.23383

In [137]:
pred_e2d2

array([[[ 1.34764338e+00,  2.06637487e-01]],

       [[ 1.50211692e+00,  2.08470449e-01]],

       [[ 1.56148696e+00,  1.96188167e-01]],

       [[ 1.48856103e+00,  1.38958663e-01]],

       [[ 1.31000042e+00,  1.68216810e-01]],

       [[ 1.37083161e+00,  1.16881967e-01]],

       [[ 1.48410726e+00, -2.87810396e-02]],

       [[ 1.54697323e+00, -1.39211029e-01]],

       [[ 1.53585732e+00, -1.26988664e-01]],

       [[ 1.62768471e+00, -1.51648581e-01]],

       [[ 1.63570333e+00, -1.33248702e-01]],

       [[ 1.62885046e+00, -6.75199777e-02]],

       [[ 1.67410803e+00,  6.44800514e-02]],

       [[ 1.69279957e+00,  1.41467348e-01]],

       [[ 1.68664384e+00,  1.19610712e-01]],

       [[ 1.66494012e+00,  7.20531493e-02]],

       [[ 1.64818335e+00,  6.24834895e-02]],

       [[ 1.59705186e+00,  2.48614866e-02]],

       [[ 1.53034091e+00, -3.97101045e-02]],

       [[ 1.50703454e+00, -9.35275853e-02]],

       [[ 1.43800128e+00, -1.57086059e-01]],

       [[ 1.33287287e+00, -1.74776

Trying to rescale it back:

In [118]:
pred_e1d1

array([[[0.8273743 , 0.79370624]],

       [[1.1508656 , 0.9567325 ]],

       [[1.254901  , 0.97861695]],

       [[1.0909071 , 0.83264905]],

       [[0.8206133 , 0.84674746]],

       [[0.93801266, 0.85351646]],

       [[1.1290182 , 0.6161981 ]],

       [[1.2246875 , 0.42144448]],

       [[1.2059498 , 0.4942795 ]],

       [[1.3912245 , 0.4522264 ]],

       [[1.3902321 , 0.48341095]],

       [[1.3822825 , 0.59960073]],

       [[1.4832363 , 0.8824918 ]],

       [[1.494795  , 0.99587184]],

       [[1.4383651 , 0.8645759 ]],

       [[1.369577  , 0.701273  ]],

       [[1.3411762 , 0.6633656 ]],

       [[1.2458911 , 0.56537974]],

       [[1.1354998 , 0.41738775]],

       [[1.1302475 , 0.31868386]],

       [[1.0371    , 0.19750166]],

       [[0.912163  , 0.17166181]],

       [[0.8669664 , 0.17970067]],

       [[0.51172286, 0.17291912]],

       [[0.5738738 , 0.29582286]],

       [[0.44921288, 0.4674527 ]],

       [[0.38344252, 0.40487832]],

       [[0.57978934, 0.41932

In [117]:
nsamples, nx, ny = train.shape
d2_train_dataset = train.reshape((nsamples,nx*ny))

ValueError: too many values to unpack (expected 3)